 <h1 align="center" style="color:#4ca8ed">Інтелектульний аналіз даних.<br>Группа: БС-81<br>Мумінов А.O.<br><br>Тема: 4.Ієрархічна кластеризація</h1>

### Завдання
1. Реалізувати ієрархічну кластеризацію.
2. Вивести таблицю об'єднання кластерів та дендрограму (побудова графіку не обов'язкова, але буде перевагою)
3. З набору даних в КП2 відібрати 20 випадкових об'єктів та провести кластерний аналіз.
4. Зробити висновки з обов'язковим вказанням оптимальної кількості кластерів.

In [2]:
import pandas as pd
import numpy as np
import random as rn
import PrData as pr

In [3]:
data = pd.read_csv('dmd.csv') #ck,age,carrier
del data['Unnamed: 0']
data = data[['age','pk','carrier']]
pr.FillGapsNum(data,'pk',3,0,'pkF')
pr.DnormalMM(data,'age','nage')
pr.DnormalMM(data,'pkF','npkF')
data

,age,pk,carrier,pkF,nage,npkF
0,27,10.798828,0,10.798828,0.170732,0.074618
1,31,11.798828,0,11.798828,0.268293,0.083946
2,22,15.000000,0,15.000000,0.048780,0.113808
3,25,15.000000,0,15.000000,0.121951,0.113808
4,26,7.000000,0,7.000000,0.146341,0.039181
...,...,...,...,...,...,...
204,32,NaN,1,15.298828,0.292683,0.116595
205,30,NaN,1,15.298828,0.243902,0.116595
206,36,NaN,1,15.298828,0.390244,0.116595
207,31,NaN,1,15.298828,0.268293,0.116595


 <h1 align="center" style="color:#4ca8ed">1.Реалізувати ієрархічну кластеризацію.</h1>

In [4]:
def Hierarchical_clustering(dt, col, k = 2, metric = pr.metric_Euclid):
    data = [dt[col].values ]
    data_index = [i for i in range(len(dt))]
    clusters = [[[i] for i in range(len(dt))]]
    for s in range(len(dt)-1):
        matrix = []
        for i in range(len(data_index)-1):
            for j in range(i+1,len(data_index)):
                matrix.append(metric(data[s][i],data[s][j]))
        min_matrix = min(matrix)
        index = matrix.index(min_matrix)
        pos = index; x = 0;
        while(True):
            if((len(data[s])-1 - x) > pos):
                pos = [(x+1) + pos,x]
                break
            pos = pos - (len(data[s])-1 - x)
            x += 1
        pos = np.sort(pos)
        clusters.append([clusters[s][i] for i in range(len(clusters[s]))])
        
        clusters[s+1][pos[0]] =[i for i in clusters[s][pos[0]]] + [ i for i in clusters[s][pos[1]]]
        del clusters[s+1][pos[1]]
        data.append([i for i in data[s]])
        data[s+1][pos[0]] = [np.mean([data[s][pos[0]][i],data[s][pos[1]][i]]) 
                             for i in range(len(data[s][0]))]
        del data[s+1][pos[1]]
        del data_index[pos[1]]
        
    return [pd.DataFrame([dt[col].iloc[j] for j in i]) for i in clusters[len(clusters)-k]]

In [5]:
cluster = Hierarchical_clustering(data[0:5],['nage','npkF','carrier'],2)
print("Три кластера")
for i in cluster:
    print("Claster:\n" + str(len(i)))
    print(i)

Три кластера
Claster:
4
       nage      npkF  carrier
0  0.170732  0.074618      0.0
4  0.146341  0.039181      0.0
3  0.121951  0.113808      0.0
2  0.048780  0.113808      0.0
Claster:
1
       nage      npkF  carrier
1  0.268293  0.083946      0.0


 <h1 align="center" style="color:#4ca8ed">2. Вивести таблицю об'єднання кластерів та дендрограму (побудова графіку не обов'язкова, але буде перевагою).</h1>

In [7]:
def Hierarchical_Visclustering(dt, col, k = 2, metric = pr.metric_Euclid):
    data = [dt[col].values ]
    data_index = [i for i in range(len(dt))]
    clusters = [[[i] for i in range(len(dt))]]
    table = pd.DataFrame(columns = ['точки','відстань','нова точка'])
    for s in range(len(dt)-1):
        matrix = []
        for i in range(len(data_index)-1):
            for j in range(i+1,len(data_index)):
                matrix.append(metric(data[s][i],data[s][j]))
        print(matrix)
        min_matrix = min(matrix)
        index = matrix.index(min_matrix)
        pos = index; x = 0;
        while(True):
            if((len(data[s])-1 - x) > pos):
                pos = [(x+1) + pos,x]
                break
            pos = pos - (len(data[s])-1 - x)
            x += 1
        pos = np.sort(pos)
        table.loc[s] = [[data_index[i] for i in pos],min_matrix,data_index[pos[0]]]
        clusters.append([clusters[s][i] for i in range(len(clusters[s]))])
        
        clusters[s+1][pos[0]] =[i for i in clusters[s][pos[0]]] + [ i for i in clusters[s][pos[1]]]
        del clusters[s+1][pos[1]]
        data.append([i for i in data[s]])
        data[s+1][pos[0]] = [np.mean([data[s][pos[0]][i],data[s][pos[1]][i]]) 
                             for i in range(len(data[s][0]))]
                
        del data[s+1][pos[1]]
        del data_index[pos[1]]
    print("Таблиця об'єднання:")
    print(table)
    for i in clusters:
        print (i)
    return [pd.DataFrame([dt[col].iloc[j] for j in i]) for i in clusters[len(clusters)-k]]


In [10]:
cluster = Hierarchical_Visclustering(data[5:20],['nage','npkF','carrier'],3)
print("\n\nТри кластера")
for i in cluster:
    print("Claster:\n" + str(len(i)))
    print(i)

[0.34197271138614116, 0.3903153900338646, 0.39035537826528444, 0.31742408977469533, 0.31767751254771043, 0.43908793600987933, 0.3911794165219377, 0.2927782368580909, 0.2701612896230548, 0.31762157892220816, 0.34149526793362683, 0.26870731453295343, 0.29439480081101316, 0.3904946782964865, 0.05533658328033069, 0.04966441321788045, 0.0415017090644103, 0.045358317473897534, 0.09820023774743042, 0.06683939429623714, 0.050046765352081315, 0.07432769583150538, 0.02439024390243902, 0.02332085303362119, 0.0805075390024479, 0.07011070600900617, 0.049002963970871054, 0.016798301484363218, 0.07354919730236459, 0.07416704992630511, 0.05101702896240598, 0.01956765278210572, 0.09869825335636156, 0.1280935333871702, 0.07769528581093767, 0.048861113844462595, 0.1221786740218995, 0.10052524671282767, 0.021462472091087453, 0.07708450261453727, 0.0786764748977228, 0.048815874126582094, 0.036365954266468936, 0.09757867358284736, 0.12398986464462874, 0.07376295769814749, 0.050747673616822364, 0.12433890986

 <h1 align="center" style="color:#4ca8ed">3. З набору даних в КП2 відібрати 20 випадкових об'єктів та провести кластерний аналіз.</h1>

In [7]:
rand = rn.randint(0,len(data)-20)
data_random = data[rand:rand+20]
print(data_random)
cluster = Hierarchical_clustering(data_random,['nage','npkF','carrier'],5)
print("Три кластера")
for i in cluster:
    print("Claster:\n" + str(len(i)))
    print(i.head(10))

    age         pk  carrier        pkF      nage      npkF
38   26  14.599609        0  14.599609  0.146341  0.110073
39   30  15.398438        0  15.398438  0.243902  0.117524
40   33  12.199219        0  12.199219  0.317073  0.087681
41   30  17.500000        0  17.500000  0.243902  0.137128
42   32  14.500000        0  14.500000  0.292683  0.109143
43   38   8.898438        0   8.898438  0.439024  0.056890
44   23  15.199219        0  15.199219  0.073171  0.115666
45   39  17.097656        0  17.097656  0.463415  0.133375
46   32   9.898438        0   9.898438  0.292683  0.066218
47   25   7.299805        0   7.299805  0.121951  0.041978
48   44  22.000000        1  22.000000  0.585366  0.179106
49   25  13.000000        0  13.000000  0.121951  0.095151
50   31  10.798828        0  10.798828  0.268293  0.074618
51   26  17.097656        0  17.097656  0.146341  0.133375
52   26  15.298828        0  15.298828  0.146341  0.116595
53   23  10.500000        0  10.500000  0.073171  0.0718

 <h1 align="center" style="color:#4ca8ed">4. Зробити висновки з обов'язковим вказанням оптимальної кількості кластерів.</h1>

Висновок: Кількість об'єктів в кластерах різна, багато маленьких кластерів, так як точки віддалені від інших і тому сполучаються в останню чергу, і при розбитті ребер утворюють малі кластера.

При невеликій кількості об'єктів, для визначення оптимальної кількості кластерів, зручно будувати дендрограму, вона дає наглядне представлення про кластера. Але при великій кількості об'єктів дендрограма буде занадто массивною для визначення оптимальної кількості кластерів. Також можно продивлятися послідовно кожен крок розбиття ребер при утворенні кластеров і таким чином аналізувати отримувані на кожній ітерації кластера, таким чином також можно підібрати оптимальну кількість кластерів. При такому підході можно розраховувати різні статистичні параметри для отриманих кластерів, для більшої єфективності.
